#### Carregar as bibliotecas

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

#### Carregar a variável de ambiente do OpenAI

In [2]:
load_dotenv()

True

#### Configuração inicial do modelo que ser utilizado

In [3]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

#### Carregar o primeiro arquivo CSV

Esse arquivo CSV possui informações sobre as propriedades de imóveis em São Paulo. Ele contém informações como endereço, nome, latitude e longitude, tipo de imóvel, distrito, etc. Essas informações são utilizadas para criar um banco de dados vetorial e armazenar as propriedades de imóveis em um banco de dados vetorial.

In [4]:
loader_properties = CSVLoader("/Users/victor/Desktop/projects/previsao-precos-imoveis-dl/data/sao-paulo-properties.csv")
docs_properties = loader_properties.load()
print(len(docs_properties))

13640


Esse arquivo CSV possui informações sobre os estabelecimentos próximos dos imóveis em São Paulo

In [5]:
loader_places = CSVLoader("/Users/victor/Desktop/projects/previsao-precos-imoveis-dl/data/places_data.csv")
docs_places = loader_places.load()
print(len(docs_places))

8352


#### Carregando o modelo de embeddings que será utilizado

In [6]:
embeddings = OpenAIEmbeddings()

#### Unido as duas bases de dados em uma única lista de documentos

In [7]:
data = docs_properties + docs_places

Verificando os 10 primeiros elementos da lista

In [8]:
data[0:10]

[Document(metadata={'source': '/Users/victor/Desktop/projects/previsao-precos-imoveis-dl/data/sao-paulo-properties.csv', 'row': 0}, page_content='Price: 930\nCondo: 220\nSize: 47\nRooms: 2\nToilets: 2\nSuites: 1\nParking: 1\nElevator: 0\nFurnished: 0\nSwimming Pool: 0\nNew: 0\nDistrict: Artur Alvim/São Paulo\nNegotiation Type: rent\nProperty Type: apartment\nLatitude: -23.543138\nLongitude: -46.479486'),
 Document(metadata={'source': '/Users/victor/Desktop/projects/previsao-precos-imoveis-dl/data/sao-paulo-properties.csv', 'row': 1}, page_content='Price: 1000\nCondo: 148\nSize: 45\nRooms: 2\nToilets: 2\nSuites: 1\nParking: 1\nElevator: 0\nFurnished: 0\nSwimming Pool: 0\nNew: 0\nDistrict: Artur Alvim/São Paulo\nNegotiation Type: rent\nProperty Type: apartment\nLatitude: -23.550239\nLongitude: -46.480718'),
 Document(metadata={'source': '/Users/victor/Desktop/projects/previsao-precos-imoveis-dl/data/sao-paulo-properties.csv', 'row': 2}, page_content='Price: 1000\nCondo: 100\nSize: 48\nRo

Verificando os últimos elementos da lista

In [9]:
data[-5:]

[Document(metadata={'source': '/Users/victor/Desktop/projects/previsao-precos-imoveis-dl/data/places_data.csv', 'row': 8347}, page_content='id: ChIJDTs9i6hfzpQRb-Is73VFIs0\naddress: R. Demétrio Ribeiro, 137 - Água Rasa, São Paulo\nname: Auto Posto Petrowan\nlatitude: -23.5525894\nlongitude: -46.5776348\ngoogleMapsUri: https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce5fa88b3d3b0d:0xcd224575ef2ce26f!10e5\nwebsiteUri: \ntype: gas_station\ndistrict: Água Rasa São Paulo'),
 Document(metadata={'source': '/Users/victor/Desktop/projects/previsao-precos-imoveis-dl/data/places_data.csv', 'row': 8348}, page_content='id: ChIJmyyKzyBczpQRNbWyAoIxTqU\naddress: R. Miguel Mota, 170 - Água Rasa, São Paulo\nname: Petrobras\nlatitude: -23.565604\nlongitude: -46.5698324\ngoogleMapsUri: https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce5c20cf8a2c9b:0xa54e318202b2b535!10e5\nwebsiteUri: https://www.postospetrobras.com.br/buscar-postos\ntype: gas_station\ndistrict: Água Rasa São Paulo'),
 Docum

#### Confirmando o formato da lista para que possamos inserir dentro do banco de dados vetorial

In [10]:
type(data)

list

#### Banco de dados vetorial - FAISS

In [11]:
vectordb = FAISS.from_documents(data, embeddings)
retriever = vectordb.as_retriever()

#### Criação do prompt template para ser utilizado na conversa

In [12]:
prompt = ChatPromptTemplate.from_template(
    """ 
    Você é um assistente especializado em recomendação de imóveis com base na proximidade de estabelecimentos relevantes. Sempre mencione uma lista de imóveis recomendados.
    Há duas bases de dados:
    - **Base de dados sobre Imóveis**: Contém informações sobre os imóveis disponíveis, como número de quartos, vagas de garagem, tamanho, e localização.
    - **Base de dados sobre Estabelecimentos**: Contém a localização de escolas, farmácias, hospitais, supermercados e outros pontos de interesse.

    **Instruções:**
    - Utilize apenas as informações dos arquivos CSV carregados para responder.
    - Priorize a proximidade entre estabelecimentos e imóveis para sugerir a melhor opção.
    - Se o usuário mencionar necessidades específicas (como ter filhos pequenos), priorize escolas e farmácias próximas.
    - Caso não encontre informações suficientes, informe que não sabe.

    **Contexto fornecido:**
    {context}

    **Pergunta do usuário:**
    {input}
    """
    )

#### Chain

In [13]:
documents_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, documents_chain)

#### Função para que o usuário possa interagir com a base de dados

In [14]:
def ask_real_estate_question(question: str):
  response = retrieval_chain.invoke({"input": question})
  print(response["answer"]) 

In [15]:
ask_real_estate_question("Tenho um filho pequeno e procuro um apartamento com escolas por perto na região da Vila Mariana.")

Para atender à sua necessidade de encontrar um apartamento na Vila Mariana com escolas próximas, aqui estão algumas opções de escolas na região:

1. **Escola Infantil Pequenos Pensadores**
   - Endereço: R. Dr. Fabrício Vampré, 302 - Vila Mariana, São Paulo
   - [Google Maps](https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce59860a301f37:0x8e69ce4d1c96fee9!10e5)
   - [Website](https://www.escolapequenospensadores.com.br/)

2. **Escola do Bairro**
   - Endereço: R. Joaquim Távora, 1548 - e 1568 - Vila Mariana, São Paulo
   - [Google Maps](https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce598bcba5f0ff:0xdc073a5ede24ebaa!10e5)
   - [Website](http://www.escoladobairro.com/)

3. **Colégio Batista Brasil Vila Mariana**
   - Endereço: R. Joaquim Távora, 618 - Vila Mariana, São Paulo
   - [Google Maps](https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce5986fa049333:0xd23fd2562f247258!10e5)
   - [Website](https://colegiobatistabrasil.com/)

4. **Escola Shekinah - Unidade Vil

In [16]:
ask_real_estate_question("""Preciso de um apartamento para minha família com 2 filhos pequenos. Temos 2 carros então precisamos ter um imóvel com espaço para ambos os carros. 
                         Qual seria a sua recomendação de até 300.000 reais?""")

Com base nas suas necessidades de ter um apartamento para uma família com 2 filhos pequenos e espaço para 2 carros, aqui estão as opções de imóveis disponíveis que se encaixam no seu orçamento de até 300.000 reais:

1. **Apartamento em Parque do Carmo/São Paulo**
   - **Preço:** R$ 225.000
   - **Condomínio:** R$ 300
   - **Tamanho:** 50 m²
   - **Quartos:** 2
   - **Suítes:** 1
   - **Vagas de Garagem:** 1
   - **Elevador:** Não
   - **Mobiliado:** Sim
   - **Tipo de Negociação:** Venda

2. **Apartamento em Parque do Carmo/São Paulo**
   - **Preço:** R$ 220.000
   - **Condomínio:** R$ 300
   - **Tamanho:** 50 m²
   - **Quartos:** 2
   - **Suítes:** 1
   - **Vagas de Garagem:** 1
   - **Elevador:** Não
   - **Mobiliado:** Não
   - **Tipo de Negociação:** Venda

Infelizmente, ambos os apartamentos listados têm apenas 1 vaga de garagem, o que não atende à sua necessidade de espaço para 2 carros. 

Infelizmente, não encontrei imóveis que atendam a todas as suas necessidades específicas. S

In [17]:
ask_real_estate_question("""Trabalho nas coordenadas latitude: -23.5660935, longitude: -46.6372631. 
                         Qual seria a melhor recomendação de imóvel considerando que meus filhos estudam nas coordenadas latitude: -23.560846, longitude: -46.604266 e
                         minha esposa trabalha em latitude: -23.525464, longitude: -46.562212""")

Para encontrar a melhor recomendação de imóvel considerando a proximidade das escolas e do local de trabalho da sua esposa, vamos analisar as informações disponíveis.

### Estabelecimentos relevantes:
1. **Escolas**:
   - **Coala Kids - Berçário - Escola Infantil - Sistema de Ensino Objetivo**
     - Endereço: Rua Kaoru Oda, 43 - Vila Sonia, São Paulo
     - Coordenadas: -23.5857802, -46.7408414
     - [Google Maps](https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce5687714183af:0x9846b2ef86243e58!10e5)

   - **Escola de Educação Infantil Girafinha Kids**
     - Endereço: R. Dias Vieira, 313 - Vila Sonia, São Paulo
     - Coordenadas: -23.5965581, -46.7318895
     - [Google Maps](https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce57e59c30376f:0xeec6d42f154c5562!10e5)

2. **Local de trabalho da sua esposa**:
   - Coordenadas: -23.525464, -46.562212

### Análise de imóveis disponíveis:
- **Casa do Norte Lageado**
  - Endereço: Estr. do Lageado Velho, 752 - Guaianases, São Pau

## Testes com Personas

### Persona 1 - João, o profissional com família pequena

In [18]:
ask_real_estate_question("""
                         Tenho 1 filha, que estuda na escola que fica em (lat:-23.560846, lon:-46.604266), minha esposa trabalha em (lat:-23.5660935, lon:-46.6372631) e eu trabalho remotamente, 
                         então preciso de um ambiente silencioso e com boa conexão de internet. A casa deve ter pelo menos 3 quartos para que minha filha tenha um espaço próprio 
                         e eu possa montar um escritório. Também preciso de pelo menos 2 banheiros para mais conforto. Temos 1 carro, então um posto de gasolina próximo seria ideal. 
                         Além disso, buscamos um bairro seguro e com mercado e farmácia a uma distância acessível. Gostamos de praticar esportes, então a presença de uma academia e 
                         um parque na região é essencial.
                         """)

Com base nas suas necessidades, aqui estão algumas recomendações de imóveis que atendem aos seus critérios:

1. **Casa Nova**
   - **Endereço:** Av. João dos Santos Abreu, 613 - Vila Espanhola, São Paulo
   - **Quartos:** 3 (ideal para sua filha e escritório)
   - **Banheiros:** 2 (conforto para a família)
   - **Proximidade de Estabelecimentos:**
     - **Escola:** Vila dos Baixinhos - Escola de Educação Infantil (Rua Dr. SÍlvio Dante Bertacchi, 644 - Vila Sônia) - [Google Maps](https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce569424117851:0xccdaf5d588be923e!10e5)
     - **Mercado:** Casa Nova (mercado próximo)
     - **Farmácia:** Não especificada, mas a região possui opções.
     - **Academia:** Estação Vila Sônia Fitness (Rua Guanás, 690 - Vila Sonia) - [Google Maps](https://www.google.com/maps/place//data=!4m3!3m2!1s0x94ce569429d407a3:0xef51a5e3d9d395a7!10e5)
   - **Posto de Gasolina:** Verifique na região, mas há opções nas proximidades.
   - **Segurança:** Bairro conside

### Persona 2 - Mariana, a jovem profissional independente

In [19]:
ask_real_estate_question("""
                         Sou solteira e trabalho em (lat: -23.563738, lon: -46.653761), então preciso de um imóvel bem localizado, preferencialmente perto de transporte público ou com vaga de garagem para o meu carro. Como trabalho muitas horas por dia, 
                         priorizo um apartamento compacto, mas funcional, com 1 ou 2 quartos e 1 banheiro. Não cozinho muito, então gostaria que houvesse restaurantes e mercados próximos. Também me preocupo com minha saúde, então uma academia próxima seria um diferencial.
                         Nos fins de semana, gosto de sair para cafés e eventos culturais, então morar em um bairro com opções de lazer e vida noturna seria ideal.
                         """)

Com base nas suas necessidades e preferências, aqui estão algumas opções de imóveis que podem atender ao seu perfil:

### Imóveis Recomendados:

1. **Apartamento em Sacomã**
   - **Preço:** R$ 1.000
   - **Condomínio:** R$ 0
   - **Tamanho:** 35 m²
   - **Quartos:** 1
   - **Banheiros:** 2
   - **Suites:** 1
   - **Vagas de Garagem:** 1
   - **Elevador:** Não
   - **Piscina:** Sim
   - **Localização:** Sacomã/São Paulo
   - **Proximidade:** Verifique a proximidade de transporte público, restaurantes e academias.

2. **Apartamento em Sacomã**
   - **Preço:** R$ 1.000
   - **Condomínio:** R$ 455
   - **Tamanho:** 70 m²
   - **Quartos:** 1
   - **Banheiros:** 2
   - **Suites:** 1
   - **Vagas de Garagem:** 1
   - **Elevador:** Sim
   - **Piscina:** Não
   - **Localização:** Sacomã/São Paulo
   - **Proximidade:** Verifique a proximidade de transporte público, restaurantes e academias.

3. **Apartamento em Saúde**
   - **Preço:** R$ 1.000
   - **Condomínio:** R$ 425
   - **Tamanho:** 63 m²
